In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score
from sklearn.preprocessing import Normalizer,MinMaxScaler,StandardScaler,normalize
from sklearn.cross_validation import train_test_split

In [162]:
pd.set_option('max_columns',1000)

In [163]:
X = pd.read_csv('../merged_caselevel_data.csv')

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (10,14,17,235,241,249,250,265,370,375,376,386) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [164]:
X.head()

,Unnamed: 0,dissentdummy,dissent,citation,docketnumber,circuit,date,vol,beginpg,casenum,fileid,year,month,day,cite,endopin,endpage,docnum,multdoc,method,state,district,origin,source,distjudg,applfrom,adminrev,priorpub,opinstat,treat,classact,crossapp,majvotes,concur,counsel1,counsel2,amicus,sanction,initiate,interven,numappel,appnatpr,appbus,appnonp,appfed,appsubst,appstate,appfiduc,ap_stid,genapel1,bank_ap1,genapel2,bank_ap2,appel1,appel2,realapp,numresp,r_natpr,r_bus,r_nonp,r_fed,r_subst,r_state,r_fiduc,r_stid,respond1,respond2,realresp,genresp1,genresp2,bank_r1,bank_r2,const1,const2,usc1,usc1sect,usc2,usc2sect,civproc1,civproc2,crmproc1,crmproc2,juris,statecl,standing,mootness,exhaust,timely,immunity,frivol,polquest,oththres,late,frivapp,othappth,constit,fedlaw,procedur,typeiss,prejud,insane,improper,juryinst,othjury,deathpen,sentence,indict,confess,search,othadmis,plea,counsel,rtcouns,suffic,indigent,entrap,procdis,othcrim,dueproc,execord,stpolicy,weightev,pretrial,trialpro,post_trl,attyfee,judgdisc,altdisp,injunct,summary,fedvst,foreign,int_law,st_v_st,discover,othcivil,subevid,denovo,erron,capric,abusedis,judrev,genstand,notice,alj,agen_acq,freeinfo,comment,record,diverse,whlaws,casetyp1,geniss,direct1,casetyp2,direct2,habeas,decuncon,unan,signed,codej1,codej2,j2vote1,j2vote2,j2maj1,j2maj2,codej3,j3vote1,j3vote2,j3maj1,j3maj2,codej4,j4vote1,j4vote2,j4maj1,j4maj2,codej5,j5vote1,j5vote2,j5maj1,j5maj2,codej6,j6vote1,j6vote2,j6maj1,j6maj2,codej7,j7vote1,j7vote2,j7maj1,j7maj2,codej8,j8vote1,j8vote2,j8maj1,j8maj2,codej9,j9vote1,j9vote2,j9maj1,j9maj2,codej10,j10vote1,j10vote2,j10maj1,j10maj2,codej11,j11vote1,j11vote2,j11maj1,j11maj2,codej12,j12vote1,j12vote2,j12maj1,j12maj2,codej13,j13vote1,j13vote2,j13maj1,j13maj2,codej14,j14vote1,j14vote2,j14maj1,j14maj2,codej15,j15vote1,j15vote2,j15maj1,j15maj2,pagelgth,j16maj1,j16vote1,citevol,decision_date,alst_code,nofirmap,noattyap,aatty_first_name,aatty_last_name,afirm_name,amicusapp,a_groups,afirst_listed_amicus_group,r_appsubst,rlst_code,nofirmrp,noattyrp,ratty_first_name,ratty_last_name,amicusresp,r_groups,rname_of_first_listed_amicus_gro,isspty,dissmer,concmer1,concmer2,disspro,concpro1,concpro2,fedlaw2,keytotal,threshgen,turnonthresh,rfirm_namew,decisiondatenew2,dropflag,j1name,j1party,j1appres,j1distjudge,j1sample,j1score,moved,j2name,j2party,j2appres,j2distjudge,j2sample,j2score,j3name,j3party,j3appres,j3distjudge,j3sample,j3score,sumappress,divided,oldj1party,oldj2party,oldj3party,sumparty,divided1,sumpartyappt,dividedA,districtjudgeinpanel,nomoversinpanel,concurdummy,unanimous,quarter,fromdistrictcourt,typeiss2,geniss2,geniss3,yearquarter,decade2,decade,distance,quartertoelect,semiannumtoelect,lastquarter,sixmonthsbefore,sixmonthsafter,lengthopin,Wopinionlenght,totalcites,negativecites,propneg,Wtotalcites,Wlengthopin,lastthreequarter,dissmerdummy,dissprodummy,disspromerdummy,disspro0merdummy,diss0promerdummy,diss0pro0merdummy,recent,type1,type2,type3,type4,type5,type6,type7,affirmdummy,reversedummy,initiate1,initiate2,initiate3,initiate4,initiate5,concmerdummy,concprodummy,concpromerdummy,concpro0merdummy,conc0promerdummy,conc0pro0merdummy,month911_f4,month911_f3,month911_f2,month911_f1,month911_t0,month911_t1,month911_t2,month911_t3,month911_t4,month911_t5,month911_t6,month911_3mon,last3,ww2,korean,vietnam,gulf,afghan,war,clinton,caseload,decisiondate,sanctions,astate_code,afirms,aatty,aattyfirstname,aattylastname,aamicusbriefs,afirstlistedamicusgroup,genappel1,genappel2,statecode,rfiduc,rfirms,ratty,ramicusbrfs,rnameoffirstlistedamicusgroup,rcounselcode,isstyp,procedure,keynotes,constnote1,constnote2,turnthresh,otherapp,justyinst,inadqecounsel,intlaw,capricious,erroneous,yearapj,j1gender,mergej1,j2gender,mergej2,j3gender,mergej3,j1moved,j2moved,j3moved,tag,caseid
0,262,0,0,842 F.2d 556,8602027,1,1988-03-28 00:00:00,842,556,1,NaN,1988,3,28,0842/0556,564,564,86002027,NaN,7,52,7,1,1,10101,3,15,0773f2d01,1,6,0,0,3,0,5,4,0,4,2,0,3,0,0,0,0,0,3,0,52,5,2,5,2,56003,56003,0,1,1,0,0,0,0

In [90]:
X['dissentdummy'].dtypes=='int64'

True

In [91]:
del_col=[]
for i in X.columns:
    if X[i].dtypes!='int64' and X[i].dtypes!='float64':
        del_col.append(i)

In [92]:
len(del_col)

11

In [93]:
X.drop(labels=del_col,axis=1,inplace=True)

In [94]:
X.head()

,Unnamed: 0,dissentdummy,dissent,docketnumber,circuit,vol,beginpg,casenum,fileid,year,month,day,endopin,endpage,docnum,multdoc,method,state,district,origin,source,distjudg,applfrom,adminrev,opinstat,treat,classact,crossapp,majvotes,concur,counsel1,counsel2,amicus,sanction,initiate,interven,numappel,appnatpr,appbus,appnonp,appfed,appsubst,appstate,appfiduc,ap_stid,genapel1,bank_ap1,genapel2,bank_ap2,appel1,appel2,realapp,numresp,r_natpr,r_bus,r_nonp,r_fed,r_subst,r_state,r_fiduc,r_stid,respond1,respond2,realresp,genresp1,genresp2,bank_r1,bank_r2,const1,const2,usc1,usc1sect,usc2,usc2sect,civproc1,civproc2,crmproc1,crmproc2,juris,statecl,standing,mootness,exhaust,timely,immunity,frivol,polquest,oththres,late,frivapp,othappth,constit,fedlaw,procedur,typeiss,prejud,insane,improper,juryinst,othjury,deathpen,sentence,indict,confess,search,othadmis,plea,counsel,rtcouns,suffic,indigent,entrap,procdis,othcrim,dueproc,execord,stpolicy,weightev,pretrial,trialpro,post_trl,attyfee,judgdisc,altdisp,injunct,summary,fedvst,foreign,int_law,st_v_st,discover,othcivil,subevid,denovo,erron,capric,abusedis,judrev,genstand,notice,alj,agen_acq,freeinfo,comment,record,diverse,whlaws,casetyp1,geniss,direct1,casetyp2,direct2,habeas,decuncon,unan,signed,codej1,codej2,j2vote1,j2vote2,j2maj1,j2maj2,codej3,j3vote1,j3vote2,j3maj1,j3maj2,codej4,j4vote1,j4vote2,j4maj1,j4maj2,codej5,j5vote1,j5vote2,j5maj1,j5maj2,codej6,j6vote1,j6vote2,j6maj1,j6maj2,codej7,j7vote1,j7vote2,j7maj1,j7maj2,codej8,j8vote1,j8vote2,j8maj1,j8maj2,codej9,j9vote1,j9vote2,j9maj1,j9maj2,codej10,j10vote1,j10vote2,j10maj1,j10maj2,codej11,j11vote1,j11vote2,j11maj1,j11maj2,codej12,j12vote1,j12vote2,j12maj1,j12maj2,codej13,j13vote1,j13vote2,j13maj1,j13maj2,codej14,j14vote1,j14vote2,j14maj1,j14maj2,codej15,j15vote1,j15vote2,j15maj1,j15maj2,pagelgth,j16maj1,j16vote1,citevol,decision_date,alst_code,nofirmap,noattyap,aatty_first_name,aatty_last_name,afirm_name,amicusapp,a_groups,afirst_listed_amicus_group,r_appsubst,rlst_code,nofirmrp,noattyrp,ratty_first_name,ratty_last_name,amicusresp,r_groups,rname_of_first_listed_amicus_gro,isspty,dissmer,concmer1,concmer2,disspro,concpro1,concpro2,fedlaw2,keytotal,threshgen,turnonthresh,rfirm_namew,decisiondatenew2,dropflag,j1party,j1appres,j1distjudge,j1sample,j1score,moved,j2party,j2appres,j2distjudge,j2sample,j2score,j3party,j3appres,j3distjudge,j3sample,j3score,sumappress,divided,oldj1party,oldj2party,oldj3party,sumparty,divided1,sumpartyappt,dividedA,districtjudgeinpanel,nomoversinpanel,concurdummy,unanimous,quarter,fromdistrictcourt,typeiss2,geniss2,geniss3,yearquarter,decade2,decade,distance,quartertoelect,semiannumtoelect,lastquarter,sixmonthsbefore,sixmonthsafter,lengthopin,Wopinionlenght,totalcites,negativecites,propneg,Wtotalcites,Wlengthopin,lastthreequarter,dissmerdummy,dissprodummy,disspromerdummy,disspro0merdummy,diss0promerdummy,diss0pro0merdummy,recent,type1,type2,type3,type4,type5,type6,type7,affirmdummy,reversedummy,initiate1,initiate2,initiate3,initiate4,initiate5,concmerdummy,concprodummy,concpromerdummy,concpro0merdummy,conc0promerdummy,conc0pro0merdummy,month911_f4,month911_f3,month911_f2,month911_f1,month911_t0,month911_t1,month911_t2,month911_t3,month911_t4,month911_t5,month911_t6,month911_3mon,last3,ww2,korean,vietnam,gulf,afghan,war,clinton,caseload,decisiondate,sanctions,astate_code,afirms,aatty,aattyfirstname,aattylastname,aamicusbriefs,afirstlistedamicusgroup,genappel1,genappel2,statecode,rfiduc,rfirms,ratty,ramicusbrfs,rnameoffirstlistedamicusgroup,rcounselcode,isstyp,procedure,keynotes,constnote1,constnote2,turnthresh,otherapp,justyinst,inadqecounsel,intlaw,capricious,erroneous,yearapj,j1gender,j2gender,j3gender,j1moved,j2moved,j3moved,tag
0,262,0,0,8602027,1,842,556,1,NaN,1988,3,28,564,564,86002027,NaN,7,52,7,1,1,10101,3,15,1,6,0,0,3,0,5,4,0,4,2,0,3,0,0,0,0,0,3,0,52,5,2,5,2,56003,56003,0,1,1,0,0,0,0,0,0,0,73700,99999,0,7,9,2,9,108,114,42,1983,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [95]:
y=X['dissentdummy'] #should be called 'label'. this is dummy

In [96]:
X.drop(labels=['dissentdummy'],axis=1,inplace=True) #drop the 'label' col

In [98]:
#fill na's
X.fillna(0,inplace=True)

In [99]:
X=X.values
y=y.values

In [131]:
X_tr,X_te,y_tr,y_te=train_test_split(X,y,test_size=0.3,random_state=42)

In [132]:
print X_tr.shape
print y_tr.shape
print X_te.shape
print y_te.shape

(700, 400)
(700,)
(300, 400)
(300,)


In [133]:
##normalize train/test data
scaler = StandardScaler()
scaler.fit(X_tr)
print X_te[:10]
print x_mean.shape
X_te=scaler.transform(X_te)
print X_te[:10]

[[  1.66690000e+04   0.00000000e+00   8.60182200e+06 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  8.40900000e+03   0.00000000e+00   8.50105900e+06 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  9.88300000e+03   0.00000000e+00   8.50170800e+06 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  1.16170000e+04   0.00000000e+00   8.50290400e+06 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.45510000e+04   0.00000000e+00   8.50576600e+06 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.88400000e+03   1.00000000e+00   8.70346600e+06 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
(400,)
[[ 1.68256718 -0.33333333  0.2467548  ..., -0.21526221 -0.16703337  0.        ]
 [ 0.02323609 -0.33333333 -0.594205   ..., -0.21526221 -0.16703337  0.        ]
 [ 0.31934432 -0.33333333 -0.5887885  ..., -0.21526221 -0.16703337  0.        ]
 ..., 
 [ 0.66768332 -0.33333333 -0.57880678 ..., -0.21526221 -0.16703337

In [151]:
paramgrid={'n_estimators':[10,50,100],'max_depth':[1,5,10,15]}

In [152]:
rf_clf=RandomForestClassifier(random_state=42)

In [153]:
scorer=make_scorer(accuracy_score)

In [154]:
%%time
gridclf=GridSearchCV(rf_clf,paramgrid,scoring=scorer,cv=3,verbose=2)

In [155]:
gridclf.fit(X_tr,y_tr)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] n_estimators=10, max_depth=1 ....................................
[CV] ........................... n_estimators=10, max_depth=1 -   0.0s
[CV] n_estimators=10, max_depth=1 ....................................
[CV] ........................... n_estimators=10, max_depth=1 -   0.0s
[CV] n_estimators=10, max_depth=1 ....................................
[CV] ........................... n_estimators=10, max_depth=1 -   0.0s
[CV] n_estimators=50, max_depth=1 ....................................
[CV] ........................... n_estimators=50, max_depth=1 -   0.1s
[CV] n_estimators=50, max_depth=1 ....................................
[CV] ........................... n_estimators=50, max_depth=1 -   0.1s
[CV] n_estimators=50, max_depth=1 ....................................
[CV] ........................... n_estimators=50, max_depth=1 -   0.1s
[CV] n_estimators=100, max_depth=1 ...................................
[CV] ...........

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:    3.8s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_estimators': [10, 50, 100], 'max_depth': [1, 5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring=make_scorer(accuracy_score), verbose=2)

In [157]:
print gridclf.best_params_
print gridclf.best_score_

{'n_estimators': 50, 'max_depth': 10}
1.0


In [158]:
gridclf.predict(X_te)

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0])

In [159]:
y_te_pred=gridclf.predict(X_te)

In [160]:
accuracy_score(y_te,y_te_pred)

0.91666666666666663